# Direct sampling with Gaussian process regression

#### Import general modules

mpi4py is always required when using these tools. Numpy is always good to have if any manipulation is to be done.

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np

# Get mpi info
comm = MPI.COMM_WORLD

#### Import modules from pynektools

In this case we will import all the data types that we currently support, as well as io functions that are required to populate them.

In [ ]:
# Data types
from pynektools.datatypes.msh import Mesh
from pynektools.datatypes.coef import Coef
from pynektools.datatypes.field import Field, FieldRegistry

# Readers
from pynektools.io.ppymech.neksuite import preadnek, pynekread

# Writers
from pynektools.io.ppymech.neksuite import pwritenek, pynekwrite

fname = '/home/adperez/cpc_gaussian_process_compression/data/mixlay/mixlay0.f00001'
#fname = '/home/adperez/Documents/gaussian_process/Gaussian Process_0823/data/turbPipe/turbPipe0.f00001'


## Read the data

In [3]:
msh = Mesh(comm, create_connectivity=False)
fld = FieldRegistry(comm)

pynekread(fname, comm, data_dtype=np.double, msh = msh, fld = fld)


for e in range(msh.nelv):
    if (np.min(msh.x[e]), np.max(msh.x[e])) == (0, 0.25):
        if (np.min(msh.y[e]), np.max(msh.y[e])) == (0, 1.1399999856948853):
            print(e)
    
coef = Coef(msh=msh, comm=comm)

2025-02-19 20:05:16,605 - Mesh - INFO - Initializing empty Mesh object.
2025-02-19 20:05:16,607 - Field - INFO - Initializing empty Field object
2025-02-19 20:05:16,608 - pynekread - INFO - Reading file: /home/adperez/Documents/gaussian_process/Gaussian Process_0823/data/turbPipe/turbPipe0.f00001
2025-02-19 20:05:16,932 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2025-02-19 20:05:16,933 - Mesh - INFO - Initializing common attributes.
2025-02-19 20:05:16,934 - Mesh - INFO - Getting vertices
2025-02-19 20:05:16,950 - Mesh - INFO - Getting edge centers
2025-02-19 20:05:17,584 - Mesh - INFO - Getting facet centers
2025-02-19 20:05:18,117 - Mesh - INFO - Mesh object initialized.
2025-02-19 20:05:18,118 - Mesh - INFO - Mesh data is of type: float64
2025-02-19 20:05:18,118 - Mesh - INFO - Elapsed time: 1.18616782s
2025-02-19 20:05:18,119 - pynekread - INFO - Reading field data
2025-02-19 20:05:18,579 - pynekread - INFO - File read
2025-02-19 20:05:18,580 - pynekread - INFO -

## Initialize the Direct sampler

In [4]:
from pynektools.compression.gpc_direct_sampling import DirectSampler

ds = DirectSampler(comm=comm, msh=msh)

In [5]:
#ds.estimate_covariance("average", elements_to_average=100)
#ds.sample(method="fixed_bitrate", n_samples=63)

n_samples = 30
bitrate = n_samples/(msh.lx*msh.ly*msh.lz)
ds.sample_field(field=fld.registry["u"], field_name="u", covariance_method="average", covariance_elements_to_average=int(msh.nelv/10), compression_method="fixed_bitrate", bitrate = bitrate)


2025-02-19 20:05:26,860 - DirectSampler - INFO - Sampling the field with options: covariance_method: {covariance_method}, compression_method: {compression_method}
2025-02-19 20:05:26,861 - DirectSampler - INFO - Estimating the covariance matrix
2025-02-19 20:05:26,862 - DirectSampler - INFO - Transforming the field into to legendre space
2025-02-19 20:05:32,758 - DirectSampler - INFO - Estimating the covariance matrix using the averaging method method. Averaging over 3780 elements at a time
2025-02-19 20:05:32,778 - DirectSampler - INFO - Covariance saved in field data_to_compress["u"]["kw"]
2025-02-19 20:05:32,779 - DirectSampler - INFO - Sampling the field using the fixed bitrate method. using settings: {'method': 'fixed_bitrate', 'bitrate': np.float64(0.05859375), 'n_samples': 30}
0
(10, 3780, 1, 512)
(1, 1, 512, 512)


MemoryError: Unable to allocate 73.8 GiB for an array with shape (10, 3780, 512, 512) and data type float64

In [ ]:
# Hack it a bit to make the rest work

ds.field = fld.registry["u"]
ds.field_sampled = ds.data_to_compress["u"]["field"]
ds.kw = ds.data_to_compress["u"]["kw"]
ds.elements_to_average = ds.settings["covariance"]["elements_to_average"]
ds.n_samples = ds.settings["compression"]["n_samples"]

In [ ]:
ind_50 = np.where(ds.field_sampled[0] == -50)
#print(ds.field[0])
#print(ds.field_sampled[0])
a = abs(ds.field_sampled[0])-abs(ds.field[0])
a[ind_50] = -50
print(a)

In [ ]:

print(ds.field_hat[1466].reshape(-1,1, order='F'))

print("======")

print(ds.field_sampled[1466].reshape(-1,1, order='F'))

In [ ]:
recon = ds.predict(ds.field_sampled)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), dpi = 200)
c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() ,fld.registry["u"].flatten(), levels=100, cmap="RdBu_r")
fig.colorbar(c)
ax.set_aspect('equal')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([1.5,6.5])
ax.set_ylim([5.5,8.5])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), dpi = 200)
#c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() , fld.registry["u"].flatten() - recon.flatten(), levels=100, cmap="RdBu_r")
c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() ,recon.flatten(), levels=100, cmap="RdBu_r")
fig.colorbar(c)
ax.set_aspect('equal')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([1.5,6.5])
ax.set_ylim([5.5,8.5])
plt.show()

In [ ]:
print(np.mean(fld.registry["u"] - recon))